In [1]:
# SOLUTION
import numpy as np
import torch
from src.architecture import load_model, NUM_CLASSES, display_image

# load the data
device = 'cpu'
sol_test_samples = np.load('src/images.npy')
sol_test_labels = np.load('src/labels.npy')
model = load_model('src/mysterious_model.pth', device=device)

Model loaded from src/mysterious_model.pth


c:\Users\yigit\Desktop\ictf24-challenges\high-challenges\MysteriousModel\src\architecture.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filepa

In [ ]:
get_model_output_probs_on_image(model, sol_test_samples[0], device)

In [ ]:
def get_model_confidence_on_samples(model, samples, labels, device='cpu'):
    model.eval()
    total_labels = np.unique(labels).shape[0]
    confidences_per_class = {l:[] for l in range(total_labels)}
    selected_logit_per_class = {l:[] for l in range(total_labels)}
    with torch.no_grad():
        for img, lab in zip(samples,labels):
            
            output = model(torch.from_numpy(img).to(device,dtype=torch.float).unsqueeze(0))

            probs = torch.nn.functional.softmax(output.data,dim=1)
            confidences, preds = torch.max(probs, 1)
            confidences_per_class[int(lab)].append(float(confidences[0]))
            selected_logit_per_class[int(lab)].append(int(preds[0]))

    confidences_per_class_mean = np.zeros(total_labels)
    selected_logit_per_class_median = np.zeros(total_labels).astype(int)
    for l,k in confidences_per_class.items():
        confidences_per_class_mean[l] = np.mean(k)
        selected_logit_per_class_median[l] = np.median(selected_logit_per_class[l]).astype(int)

    return confidences_per_class_mean, selected_logit_per_class_median

avg_confs, med_selected_logits = get_model_confidence_on_samples(model, sol_test_samples, sol_test_labels, device='cpu')
most_confident_classes = np.argsort(avg_confs)[-NUM_CLASSES:]
most_selected_logits = med_selected_logits[most_confident_classes]
flag = most_confident_classes[np.argsort(most_selected_logits)]

print('PREDICTED:')
print(f'MOST CONFIDENT CLASSES: {most_confident_classes}')
print(f'MOST SELECTED LOGITS: {most_selected_logits}')
print(f'FLAG: {flag}')

In [ ]:
with open('sol_flag.txt', 'w') as fp:
    classes = ','.join([str(l) for l in flag])
    fp.write(f'ictf{{{classes}}}')